<a href="https://colab.research.google.com/github/Nightcrawler9x/Beyond_visible_spectrum/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Challenge

##Generalidades

In [1]:
#@title Imports
import gdown
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
#@title Funciones
def load_from_drive(file_id, name):
  gdown.download(id=file_id, output=name, quiet=False)
  return name

def zero_pad(x, target_shape):
    padded = np.zeros(target_shape, dtype=x.dtype)
    slices = tuple(slice(0, min(dim, target)) for dim, target in zip(x.shape, target_shape))
    padded[slices] = x[slices]
    return padded

In [3]:
#@title Carga de los datos
load_from_drive('1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn', 'data.zip')
!unzip "/content/data.zip" -d "/content/data"

Downloading...
From (original): https://drive.google.com/uc?id=1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn
From (redirected): https://drive.google.com/uc?id=1kVbztu0a5E3qefYhMNi7KUgzQXo12uOn&confirm=t&uuid=a4edf2e8-0a38-41c2-bfad-c28684121fcc
To: /content/data.zip
100%|██████████| 6.95G/6.95G [01:19<00:00, 87.1MB/s]


Archive:  /content/data.zip
  inflating: /content/data/ot/ot/sample1.npy  
  inflating: /content/data/ot/ot/sample10.npy  
  inflating: /content/data/ot/ot/sample100.npy  
  inflating: /content/data/ot/ot/sample1000.npy  
  inflating: /content/data/ot/ot/sample1001.npy  
  inflating: /content/data/ot/ot/sample1002.npy  
  inflating: /content/data/ot/ot/sample1003.npy  
  inflating: /content/data/ot/ot/sample1004.npy  
  inflating: /content/data/ot/ot/sample1005.npy  
  inflating: /content/data/ot/ot/sample1006.npy  
  inflating: /content/data/ot/ot/sample1007.npy  
  inflating: /content/data/ot/ot/sample1008.npy  
  inflating: /content/data/ot/ot/sample1009.npy  
  inflating: /content/data/ot/ot/sample101.npy  
  inflating: /content/data/ot/ot/sample1010.npy  
  inflating: /content/data/ot/ot/sample1011.npy  
  inflating: /content/data/ot/ot/sample1012.npy  
  inflating: /content/data/ot/ot/sample1013.npy  
  inflating: /content/data/ot/ot/sample1014.npy  
  inflating: /content/data/ot

##Exploracion de los datos

In [4]:
df_train = pd.read_csv('/content/data/train.csv')
df_test = pd.read_csv('/content/data/test.csv')

In [5]:
df_train.head()

,id,label
0,sample697.npy,7
1,sample54.npy,81
2,sample2270.npy,4
3,sample1401.npy,99
4,sample1901.npy,43


In [6]:
df_test.head()

,id
0,sample1957.npy
1,sample1369.npy
2,sample782.npy
3,sample2255.npy
4,sample2160.npy


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177 entries, 0 to 2176
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2177 non-null   object
 1   label   2177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.1+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      545 non-null    object
dtypes: object(1)
memory usage: 4.4+ KB


In [9]:
print(len(df_train['label'].unique()))

101


In [10]:
dir_csv = '/content/data/train.csv'
dir_npys = '/content/data/ot/ot'

chunk_size = 32
EXPECTED_SHAPE = (128,128,125)

In [21]:
def load_dataset(dir_csv, dir_npys, expected_shape=(128, 128, 125)):
    df = pd.read_csv(dir_csv)
    npys = []
    labels = []

    for idx, fila in df.iterrows():
        npy_path = os.path.join(dir_npys, fila['id'])

        try:
            npy = np.load(npy_path)

            if npy.shape != expected_shape:
                try:
                    npy = zero_pad(npy, expected_shape)
                except Exception as e:
                    print(f"Error haciendo padding: {npy.shape} en {e}")
                    continue

            npys.append(npy)
            labels.append(fila['label'])

        except Exception as e:
            print(f"Error cargando {npy_path}: {e}")
            continue

    X = np.array(npys)
    y = np.array(labels)

    return X, y

In [12]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00


In [22]:
X_all, y_all = load_dataset(dir_csv, dir_npys)

Error cargando /content/data/ot/ot/sample2451.npy: cannot reshape array of size 1785856 into shape (128,128,125)


In [24]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from skimage.util import view_as_windows


# === FUNCIÓN PARA EXTRAER PATCHES ===
def extract_patches_from_all(X, y, patch_size=(5, 5), stride=5):
    all_patches = []
    all_labels = []

    for volume, label in zip(X, y):
        windows = view_as_windows(volume, (patch_size[0], patch_size[1], volume.shape[2]), step=stride)
        for i in range(windows.shape[0]):
            for j in range(windows.shape[1]):
                patch = windows[i, j, 0]  # patch shape: (5, 5, 125)
                all_patches.append(patch)
                all_labels.append(label)

    return np.array(all_patches), np.array(all_labels)


# === PREPARACIÓN DE DATOS ===
# Normalizar
X_all = X_all.astype(np.float32)
X_all /= np.max(X_all)

# Extraer patches y etiquetas
X_patches, y_patches = extract_patches_from_all(X_all, y_all, patch_size=(5, 5), stride=5)

# === AUTOENCODER ===
input_shape = (5, 5, 125)
flatten_dim = np.prod(input_shape)
latent_dim = 64

input_img = Input(shape=input_shape)
x = Flatten()(input_img)
encoded = Dense(256, activation='relu')(x)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(encoded)
decoded = Dense(flatten_dim, activation='sigmoid')(decoded)
decoded_reshaped = Reshape(input_shape)(decoded)

autoencoder = Model(input_img, decoded_reshaped)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')

# Entrenar autoencoder con los patches
autoencoder.fit(X_patches, X_patches, epochs=3, batch_size=64, validation_split=0.1, verbose=1)

# Codificar los patches
X_encoded = encoder.predict(X_patches)

# === CLASIFICADOR ===
le = LabelEncoder()
y_encoded = le.fit_transform(y_patches)
y_onehot = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_onehot, test_size=0.2, random_state=42, shuffle=True
)

input_latent = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(input_latent)
x = Dense(32, activation='relu')(x)
output = Dense(y_onehot.shape[1], activation='softmax')(x)

classifier = Model(input_latent, output)
classifier.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/3
19125/19125 ━━━━━━━━━━━━━━━━━━━━ 939s 49ms/step - loss: 0.0051 - val_loss: 0.0041
Epoch 2/3
19125/19125 ━━━━━━━━━━━━━━━━━━━━ 934s 49ms/step - loss: 0.0040 - val_loss: 0.0041
Epoch 3/3
19125/19125 ━━━━━━━━━━━━━━━━━━━━ 934s 49ms/step - loss: 0.0040 - val_loss: 0.0041
42500/42500 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step
Epoch 1/3
17000/17000 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - accuracy: 0.0152 - loss: 4.6180 - val_accuracy: 0.0157 - val_loss: 4.5909
Epoch 2/3
17000/17000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.0155 - loss: 4.5908 - val_accuracy: 0.0157 - val_loss: 4.5910
Epoch 3/3
17000/17000 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.0157 - loss: 4.5909 - val_accuracy: 0.0157 - val_loss: 4.5910


In [ ]:


### Modelo Autoencoder
input_shape = (5, 5, 125)
flatten_dim = np.prod(input_shape)
latent_dim = 64

input_img = Input(shape=input_shape)
x = Flatten()(input_img)
encoded = Dense(256, activation='relu')(x)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(encoded)
decoded = Dense(flatten_dim, activation='sigmoid')(decoded)
decoded_reshaped = Reshape(input_shape)(decoded)

autoencoder = Model(input_img, decoded_reshaped)
encoder = Model(input_img, encoded)
autoencoder.compile(optimizer=Adam(1e-3), loss='mse')

### Entrenamiento
all_encoded = []
all_labels = []

for epoch in range(3):
    print(f"\n🔁 Epoch {epoch+1}/10")
    for X_chunk, y_chunk in chunk_generator(dir_csv, dir_npys):
        X_patches, y_labels = extract_patches(X_chunk, y_chunk)
        X_patches = X_patches / np.max(X_patches)

        autoencoder.fit(X_patches, X_patches, epochs=1, verbose=0)

        encoded = encoder.predict(X_patches)
        all_encoded.append(encoded)
        all_labels.extend(y_labels)

X_encoded = np.vstack(all_encoded)
patch_labels = np.array(all_labels)

# Clasificador
le = LabelEncoder()
y_encoded = le.fit_transform(patch_labels)
y_onehot = to_categorical(y_encoded)

input_latent = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(input_latent)
x = Dense(32, activation='relu')(x)
output = Dense(y_onehot.shape[1], activation='softmax')(x)

classifier = Model(input_latent, output)
classifier.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

classifier.fit(X_encoded, y_onehot, epochs=10, batch_size=64, shuffle=True, validation_split=0.2)



🔁 Epoch 1/10
